# OpenAI API: Code Interpreter and Advanced Data Analysis
## 03_01 - Generate and Apply Python Code

### Install the necessary libraries. 

In [ ]:
pip install openai

In [ ]:
pip install openai[datalib]

In [ ]:
pip install urllib3==1.26.6 

In [ ]:
pip install python-dotenv

### Import the libraries and environment file to gain access to the Open API Key
#### The key can be generated here: https://platform.openai.com/account/api-keys

In [1]:
#import OS, OpenAI, and time modules
import openai
import os
from openai import OpenAI
import time

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

### Authenticate to the API using the API Key
#### Pull from environment variables or use api_key = ("your_key_here") to hardcode the key

In [2]:
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY']  
)

### Helper function to upload Excel file

In [3]:
def upload_file(path):
    try:
        file = client.files.create(
            file=open(path, "rb"),
            purpose='assistants'
        )
        
        return file.id
    except Exception as e:
        print(e)
        return e

### Helper function to setup an Assistant
#### The Assistant is a Coding Bot
Code Interpreter allows the Assistants API to write and run Python code in a sandboxed execution environment.

In [4]:
def create_assistant(file_id):
    try:
        assistant = client.beta.assistants.create(
            name = "Coding Bot",
            instructions='''Generate a file, always. You are an expert Python developer.
                           When asked a to solve a query, you write and run code to 
                           answer the question. Make the file id available for download.''',
            model="gpt-4-1106-preview",
            tools=[{"type": "code_interpreter"}],
            file_ids=[file_id]
        )
        
        return assistant.id
    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error 

### Helper function to create a thread to query an Assistant

In [5]:
def query_assistant(assistant_id, user_query):
    try:
        thread = client.beta.threads.create(
            messages=[
                {
                "role": "user",
                "content": user_query
                }
            ]
        )
        
        return thread.id
    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error

### Helper function to create a run object to execute the thread

In [6]:
def run(assistant_id, thread_id):
    try:
        run = client.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=assistant_id
        )
        
        time.sleep(10)
        
        while True:
            print(f'{run.id=} {run.status=}')

            run = client.beta.threads.runs.retrieve(
                thread_id=thread_id,
                run_id=run.id
            )

            status = run.status

            if status == 'completed':
                break
            else:
                time.sleep(10) 

        print(f'{run.id=} {run.status=}')
        
        #retrieve the messages 
        messages = client.beta.threads.messages.list(
            thread_id=thread_id
        )

   
        return messages
        
    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error

### Read in the data file

In [7]:
file_id = upload_file('Formatted_Customer_Feedback_Product_Ratings.csv')

print(file_id)

file-0rnC4t4yrNf6sKeOp15e1e6k


### Create a run object to execute the thread

In [8]:
assistant_id = create_assistant(file_id)

In [9]:
print(assistant_id)

asst_jXeHEmhkSd4jInUTot26ya58


In [10]:
user_query = '''Can you create a heatmap image that illustrates the seasonal 
                trends in average customer ratings, segmented by month and year? 
                Supply the heatmap image as a downloadable file. Also supply any 
                Python code you generate as a downloadable file. I expect two
                downloadable files.'''

thread_id = query_assistant(assistant_id, user_query)
print(thread_id)

thread_F59H0kIlDLW5AI0UwMORrCg4


In [11]:
messages = run(assistant_id, thread_id)
print(messages)

run.id='run_eBB9oHWRk9Gfr5GFxAzxtTav' run.status='queued'
run.id='run_eBB9oHWRk9Gfr5GFxAzxtTav' run.status='in_progress'
run.id='run_eBB9oHWRk9Gfr5GFxAzxtTav' run.status='in_progress'
run.id='run_eBB9oHWRk9Gfr5GFxAzxtTav' run.status='in_progress'
run.id='run_eBB9oHWRk9Gfr5GFxAzxtTav' run.status='in_progress'
run.id='run_eBB9oHWRk9Gfr5GFxAzxtTav' run.status='in_progress'
run.id='run_eBB9oHWRk9Gfr5GFxAzxtTav' run.status='in_progress'
run.id='run_eBB9oHWRk9Gfr5GFxAzxtTav' run.status='in_progress'
run.id='run_eBB9oHWRk9Gfr5GFxAzxtTav' run.status='in_progress'
run.id='run_eBB9oHWRk9Gfr5GFxAzxtTav' run.status='completed'
SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_p8y2nA15PCICLsjYy0bmdZNE', assistant_id='asst_jXeHEmhkSd4jInUTot26ya58', content=[MessageContentText(text=Text(annotations=[TextAnnotationFilePath(end_index=165, file_path=TextAnnotationFilePathFilePath(file_id='file-oGAIGU8GHPKZqvaP9wV2Lrh2'), start_index=132, text='sandbox:/mnt/data/heatmap_code.py', type='file_path

In [12]:
#loop and print the messages out
for thread_message in messages.data:
    # Accessing the content array within each ThreadMessage
    for content in thread_message.content:
        # Checking if the content type is MessageContentText
        if content.type == 'text':
            # Accessing the text attribute of the MessageContentText
            text_content = content.text.value
            print(text_content)

The Python code used to generate the heatmap has been saved. You can download it using the link below:

[Download Python Code File](sandbox:/mnt/data/heatmap_code.py)

Please note that you will need to replace `'your_file_path.csv'` in the code with the actual path to your dataset when you want to run this code on your system.
The heatmap has been created and saved. You can download the heatmap image illustrating the seasonal trends in average customer ratings for the year 2021 using the link below:

[Download Heatmap Image](sandbox:/mnt/data/heatmap_customer_ratings.png)

Next, I will save the Python code used to generate the heatmap to a file that you can also download.
It seems that the data only contains information for the year 2021. Thus, we have average ratings for each month for that year.

With these average ratings, I will now proceed to step 3: create a heatmap, and then move on to step 4: save the heatmap image and Python code as downloadable files.
The dataset contains sev

### Download the Python code generated by Code Interpreter tool

In [13]:
for thread_message in messages.data:
    # Accessing the content array within each ThreadMessage
    for content in thread_message.content:
        # Checking if the content type is 'text'
        if content.type == 'text':
            # Accessing the annotations within the text content
            for annotation in content.text.annotations:
                # Checking for file_path type in annotations
                if annotation.type == 'file_path':
                    # Extracting the file_id
                    file_id = annotation.file_path.file_id
                    # Extracting the file path
                    file_path = annotation.text

                    # Check if the file path contains '.png'
                    if '.png' in file_path:
                        image_file_id = file_id
                        print(f"File ID: {file_id}, File Path: {file_path}")
                    elif '.py' in file_path:
                        code_file_id = file_id
                        print(f"File ID: {file_id}, File Path: {file_path}")

File ID: file-oGAIGU8GHPKZqvaP9wV2Lrh2, File Path: sandbox:/mnt/data/heatmap_code.py
File ID: file-DNd7XN5vcFAnMJ0uzsgzEsKT, File Path: sandbox:/mnt/data/heatmap_customer_ratings.png


In [14]:
print(image_file_id)

file-DNd7XN5vcFAnMJ0uzsgzEsKT


In [15]:
print(code_file_id)

file-oGAIGU8GHPKZqvaP9wV2Lrh2


In [16]:
#get file name
file_name = client.files.with_raw_response.retrieve_content(code_file_id) 
print(file_name)

<APIResponse [200 OK] type=<class 'str'>>


In [17]:
#download and save the file locally
with open("./code.py", "wb") as file:  
    file.write(file_name.content) 

### Download the Heatmap image generated by Code Interpreter tool

In [18]:
#get file name
file_name = client.files.with_raw_response.retrieve_content(image_file_id) 
print(file_name)

<APIResponse [200 OK] type=<class 'str'>>


In [19]:
#download and save the file locally
with open("./heatmap.png", "wb") as file:  
    file.write(file_name.content) 

#### Display image inline

In [20]:
#display image inline
from IPython.display import Image

image = "./heatmap.png"
print(image)

Image(url=image)

./heatmap.png


### Clean up Assistants
Do not leave Assistants or files running for longer periods

In [21]:
client.files.delete(image_file_id)

FileDeleted(id='file-DNd7XN5vcFAnMJ0uzsgzEsKT', deleted=True, object='file')

In [22]:
client.files.delete(code_file_id)

FileDeleted(id='file-oGAIGU8GHPKZqvaP9wV2Lrh2', deleted=True, object='file')

In [23]:
client.beta.assistants.delete(assistant_id)

AssistantDeleted(id='asst_jXeHEmhkSd4jInUTot26ya58', deleted=True, object='assistant.deleted')

In [24]:
client.beta.threads.delete(thread_id)

ThreadDeleted(id='thread_F59H0kIlDLW5AI0UwMORrCg4', deleted=True, object='thread.deleted')